In [91]:
import h5py

Vlist = []
with h5py.File(f"./results/tfim2d_SPARSE_103_Lx3Ly3_t0.25_layers9_U_3hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]

In [7]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm


Lx, Ly = (3, 3)
L = Lx*Ly
t = 0.2
latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 3)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

perms_v, perms_h = get_perms(Lx, Ly)
perms_extended = [perms_v, perms_v, perms_v, perms_h, perms_h, perms_h, perms_v, perms_v, perms_v]

#state = np.array(random_statevector(2**L).data)
#state = ground_state
#print("Trotter error: ", 1-state_fidelity(ansatz_sparse(Vlist, L, perms_extended, state), expm_multiply(
#    -1j * t * hamil, state)))

In [8]:
perms_v

[[0, 3, 1, 4, 2, 5], [3, 6, 4, 7, 5, 8], [6, 0, 7, 1, 8, 2]]

In [186]:
np.linalg.norm(d_phase(t))

28.319684413049284

In [187]:
from utils_sparse import applyG_state, Z
I2 = np.eye(2)
X = np.array([[0, 1+0j], [1+0j, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1+0j, 0], [0, -1+0j]])
ket_0 = np.array([[1],[0]])
ket_1 = np.array([[0],[1]])
rho_0_anc = ket_0 @ ket_0.T
rho_1_anc = ket_1 @ ket_1.T


def phase(t):
    return np.exp(-1j * t * eigenvalues_sort[0])

def d_phase(t):
    return np.exp(-1j * t * eigenvalues_sort[0]) * (-1j*eigenvalues_sort[0])


state_l = expm_multiply(-1j * t * hamil, state)
sum_ = 0
for perm in perms_v+perms_h:
    for j in range(len(perm)//2):
        state_l_ = applyG_state(np.kron(Z, Z), state_l, L, perm[2*j], perm[2*j+1])
        state_anc = np.kron(np.array([1, 0]), state_l)/np.sqrt(2) + np.kron(np.array([0, 1]), state_l_)/np.sqrt(2)
        exp = np.vdot(state_anc, np.kron(X, np.outer(ground_state, np.conj(ground_state)))@state_anc)
        sum_ += J*exp
        
for i in range(L):
    state_l_ = applyG_state(np.kron(X, np.eye(2)), state_l, L, i, (i+1)%L)
    state_anc = np.kron(np.array([1, 0]), state_l)/np.sqrt(2) + np.kron(np.array([0, 1]), state_l_)/np.sqrt(2)
    exp = np.vdot(state_anc, np.kron(X, np.outer(ground_state, np.conj(ground_state)))@state_anc)
    sum_ += g*exp

r = np.vdot(ground_state, state_l)
-sum_

(28.319684413049092-2.1510571102112408e-15j)

In [165]:
sys.path.append("../../../ccU-opt/ccU")
from utils import applyG

state_l = expm_multiply(-1j * t * hamil, ground_state)
s = 0+0j
for perm in perms_v+perms_h:
    for j in range(len(perm)//2):
        Pj = applyG(np.kron(Z, Z), perm[2*j], perm[2*j+1], L)
        ketbra = np.outer(ground_state, np.conj(ground_state))
        antc = Pj@ketbra+ketbra@Pj
        s += J*0.5*np.vdot(state_l, antc@state_l)

for i in range(L):
    Pj = applyG(np.kron(X, np.eye(2)), i, (i+1)%L, L)
    ketbra = np.outer(ground_state, np.conj(ground_state))
    antc = Pj@ketbra+ketbra@Pj
    s += g*0.5*np.vdot(state_l, antc@state_l)

-s

(28.319684413049416+1.1587952819525071e-15j)

In [166]:
"""
    Adiabatic Evolution Implementation.
"""
from qiskit import Aer, transpile, execute
from qiskit.quantum_info import random_statevector
from scipy.sparse.linalg import expm_multiply
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


def trotter(Lx, Ly, tau, L, J_i, h_i, g_i, J_f, h_f, g_f, lamb):
    L = Lx * Ly
    assert lamb <= 1 and lamb >= 0
    J = lamb*J_f + (1-lamb)*J_i
    g = lamb*g_f + (1-lamb)*g_i
    h = lamb*h_f + (1-lamb)*h_i

    qc = qiskit.QuantumCircuit(L)
    hloc = construct_ising_local_term_(J, g, 2)
        
    # permutations specifying gate layout
    # Verticals
    perms_vercs = []
    for i in range(Ly):
        start_ind = Lx*i
        perms_verc = []
        for j in range(start_ind, start_ind+Lx):
            perms_verc += [j, (j+Lx)%L]
        perms_vercs.append(perms_verc)
    if Ly == 4:
        perms_vercs = [perms_vercs[0]+perms_vercs[2], perms_vercs[1]+perms_vercs[3]]
    # Horizontals
    perms_horzs = []
    for i in range(Lx):
        start_ind = i
        perms_horz = []
        for j in range(start_ind, L, Lx):
            if start_ind != Lx-1:
                perms_horz += [j, j+1]
            else:
                perms_horz += [j, j+1-Lx]
        perms_horzs.append(perms_horz)
    if Lx == 4:
        perms_horzs = [perms_horzs[0]+perms_horzs[2], perms_horzs[1]+perms_horzs[3]]
    perm_set = perms_vercs + perms_horzs
    perms = perm_set
    
    method_start = oc.SplittingMethod.suzuki(len(perm_set), 1)
    indices = method_start.indices
    coeffs = method_start.coeffs

    Vlist_start = []
    perms = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*tau*hloc))
        perms.append(perm_set[i])
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1])
        Vlist_gates.append(qc2)
    
    for layer, qc_gate in enumerate(Vlist_gates):     
        for j in range(len(perms[layer])//2):
            qc.append(qc_gate.to_gate(), [L-(perms[layer][2*j]+1), L-(perms[layer][2*j+1]+1)])
    return qc


def construct_ising_local_term_(J, g, ndim, h=0):
    X = np.array([[0.,  1.], [1.,  0.]])
    Z = np.array([[1.,  0.], [0., -1.]])
    I = np.identity(2)
    return J*np.kron(Z, Z) + g*(0.5/ndim)*(np.kron(X, I) + np.kron(I, X)) + h*(0.5/ndim)*(np.kron(Z, I) + np.kron(I, Z))


def run_adiabatic(Lx, Ly, g, T, S, return_state=False, h_i=0, h_f=0):
    L = Lx*Ly
    tau = 1/S
    t_s = np.linspace(0, T, S*T)
    sch = lambda t, T: np.sin(np.pi*t/(2*T))**2
    
    qc = qiskit.QuantumCircuit(L)
    qc.x([i for i in range(L)])
    qc.h([i for i in range(L)])
    for s in range(S*T):
        qc.append(trotter(Lx, Ly, tau, L, 0, h_i, g, J, h_f, g, sch(t_s[s], T)).to_gate(), [i for i in range(L)])
    
    backend = Aer.get_backend("statevector_simulator")
    qc_ = qiskit.QuantumCircuit(L)
    qc_.append(qc.to_gate(), [i for i in range(L)])
    final = execute(transpile(qc_), backend).result().get_statevector().data
    print("AQC: ", [np.linalg.norm(np.vdot(final, eigenvectors_sort[:, i]))**2 for i in range(10)])

    if return_state:
        return qc
    else:
        noise_model = NoiseModel()
        dag = circuit_to_dag(transpile(qc_, basis_gates=['rzz', 'u3', 'initialize']))
        count_ops = dag.count_ops_longest_path()
        
        return [state_fidelity(final, eigenvectors_sort[:, i]) for i in range(10)], {"gates": count_ops}, final


In [167]:
J, g, h = (1, 3, 0)
H_i = qib.IsingHamiltonian(field, 0, 0., g).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(H_i, k=10)
idx = eigenvalues.argsort()
eigenvectors_sort = eigenvectors[:,idx]
state = eigenvectors_sort[:, 0]

hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
eigenvalues, eigenvectors =  scipy.sparse.linalg.eigsh(hamil, k=10)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
qc_A = run_adiabatic(Lx, Ly, g, 1, 4, return_state=True)

AQC:  [0.9742154401980777, 5.132603301791985e-32, 5.007417855406812e-32, 7.896312771987667e-31, 1.7333369499485123e-33, 1.222159957093327e-10, 4.770899325786841e-10, 7.185348533848903e-33, 2.1664171268549153e-09, 8.425568108926033e-32]


In [168]:
t = 0.25

qc_timeEvolved = qc_A.copy()
for n in range(int(t//0.25)):
    for i, V in enumerate(Vlist):
        perms = perms_extended[i]
        for perm in perms:
            for j in range(len(perm)//2):
                qc_timeEvolved.unitary(V, [L-perm[2*j]-1, L-perm[2*j+1]-1])


backend = Aer.get_backend("statevector_simulator")
sv = execute(transpile(qc_timeEvolved), backend).result().get_statevector().data
1 - state_fidelity(sv, expm_multiply(-1j * t * hamil, ground_state))

0.025987098417656118

In [172]:

sum_ = 0
for perm in perms_v+perms_h:
    for j in range(len(perm)//2):
        qc_C = qiskit.QuantumCircuit(L+1, L+1)
        qc_C.h(0)
        qc_C.append(qc_timeEvolved.to_gate(), [i for i in range(1, L+1)])
        qc_Z = qiskit.QuantumCircuit(2)
        qc_Z.unitary(np.kron(Z, Z), [0, 1])
        qc_C.append(qc_Z.to_gate().control(), [0, perm[2*j]+1, perm[2*j+1]+1])
        qc_C.h(0)
        qc_C.append(qc_A.inverse().to_gate(), [i for i in range(1, L+1)])
        qc_C.measure([i for i in range(L+1)], [i for i in range(L+1)])

        backend = qiskit.Aer.get_backend("aer_simulator")
        counts = execute(transpile(qc_C), backend, shots=1000).result().get_counts()
        exp = ((counts["0"*L+'0'] if "0"*L+'0' in counts.keys() else 0)-(counts["0"*L+'1'] if "0"*L+'1' in counts.keys() else 0))/((counts["0"*L+'1'] if "0"*L+'1' in counts.keys() else 0)+(counts["0"*L+'0'] if "0"*L+'0' in counts.keys() else 0))
        
        sum_ += J*exp


for i in range(L):
    qc_C = qiskit.QuantumCircuit(L+1, L+1)
    qc_C.h(0)
    qc_C.append(qc_timeEvolved.to_gate(), [i for i in range(1, L+1)])
    qc_C.cx(0, i+1)
    qc_C.h(0)
    qc_C.append(qc_A.inverse().to_gate(), [i for i in range(1, L+1)])
    qc_C.measure([i for i in range(L+1)], [i for i in range(L+1)])

    backend = qiskit.Aer.get_backend("aer_simulator")
    counts = execute(transpile(qc_C), backend, shots=1000).result().get_counts()
    exp = ((counts["0"*L+'0'] if "0"*L+'0' in counts.keys() else 0)-\
                   (counts["0"*L+'1'] if "0"*L+'1' in counts.keys() else 0))/\
            ((counts["0"*L+'1'] if "0"*L+'1' in counts.keys() else 0)+\
             (counts["0"*L+'0'] if "0"*L+'0' in counts.keys() else 0))

    sum_ += g*exp

r = np.linalg.norm(np.vdot(ground_state, sv))
-sum_


32.08338363737938

In [ ]:
import quimb as qu
import quimb.tensor as qtn
import numpy as np
import matplotlib.pyplot as plt

# Parameters
Lx, Ly = 4, 4
N_phys = Lx * Ly
N_total = N_phys + 1  # +1 ancilla at site 0
J = 1.0
g = 3.0
dt = 0.05
t_total = 0.2
steps = int(t_total / dt)
bond_dim = 64

# Site mapping: ancilla is site 0, physical qubits are sites 1 to 36
def phys_site(x, y):
    return 1 + x + y * Lx  # +1 due to ancilla at site 0

# Build physical qubit interaction pairs (with PBC)
pairs = []
for y in range(Ly):
    for x in range(Lx):
        i = phys_site(x, y)
        jx = phys_site(x + 1, y)
        jy = phys_site(x, y + 1)
        pairs.append((i, jx))
        pairs.append((i, jy))

pairs = list(set(tuple(sorted(p)) for p in pairs))  # deduplicate

# Create random cyclic MPS (PBC), 1 ancilla + 36 physical qubits
psi = qtn.MPS_rand_state(N_total, bond_dim=bond_dim, cyclic=True, dtype='complex128')

# 4th-order Trotter coefficients
w1 = 1.0 / (4 - 4 ** (1 / 3))
w2 = 1.0 - 2 * w1
coeffs = [w1 / 2, w1, w2, w1, w1 / 2]


def controlled_zz(anc, i, j, coeff):
    ZZ = qu.expm(-1j * dt * coeff * J * qu.pauli('Z') & qu.pauli('Z'))
    P0 = qu.ket('0') @ qu.bra('0')
    P1 = qu.ket('1') @ qu.bra('1')
    C_U = (P0 & qu.eye(4)) + (P1 & ZZ)
    return C_U.reshape(2, 2, 2, 2, 2, 2)  # (a,i,j), (a',i',j')


def controlled_x(anc, i, coeff):
    X = qu.expm(-1j * dt * coeff * g * qu.pauli('X'))
    P0 = qu.ket('0') @ qu.bra('0')
    P1 = qu.ket('1') @ qu.bra('1')
    C_U = (P0 & qu.eye(2)) + (P1 & X)
    return C_U.reshape(2, 2, 2, 2)  # (a,i), (a',i')


# Time evolution loop
anc = 0
for step in range(steps):
    for coeff in coeffs:
        # Apply controlled-ZZ
        for i, j in pairs:
            Uczz = controlled_zz(anc, i, j, coeff)
            psi.apply_two_site_gate(Uczz, (anc, i, j), contract='auto-split', max_bond=bond_dim)

        # Apply controlled-X on each physical qubit
        for i in range(1, N_total):
            Ucx = controlled_x(anc, i, coeff)
            psi.apply_two_site_gate(Ucx, (anc, i), contract='swap+split', max_bond=bond_dim)

    psi.compress(max_bond=bond_dim, cutoff=1e-10)

print(f"Final bond dimensions: {psi.bond_sizes()}")
print(f"Norm: {psi.norm():.6f}")
